In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt

from scipy.signal import argrelextrema
from scipy.stats import linregress

import configur
import ticker
import train

import tensorflow as tf

2023-08-11 15:58:03.705513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 15:58:04.327696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
repsol = ticker.Ticker("REP.MC")
if configur.data_mode == "download":
    repsol.yf_download(
        configur.start_date, 
        configur.end_date, 
        configur.interval
    )
elif configur.data_mode == "load":
    repsol.upload_data(
        configur.start_date, 
        configur.end_date
    )

repsol.get_smooth()
repsol.get_support()
repsol.get_resistance()
repsol.get_moving_average()
repsol.get_stochastic_oscillator(configur.oscillator_period)

2023-08-11 15:58:06,374 - INFO - data loaded: 2021-07-01 - 2023-06-25, shape: (4581, 6)
2023-08-11 15:58:06,375 - INFO - data smoothed, shape: (4581, 7)
2023-08-11 15:58:06,379 - INFO - extrema collected, shape: (4581, 9)


2023-08-11 15:58:06,435 - INFO - support lines collected, shape: (4581, 10)
2023-08-11 15:58:06,439 - INFO - extrema collected, shape: (4581, 10)
2023-08-11 15:58:06,493 - INFO - resistance lines collected, shape: (4581, 11)
2023-08-11 15:58:06,494 - INFO - data smoothed, shape: (4581, 11)
2023-08-11 15:58:06,496 - INFO - moving averages collected, shape: (4581, 13)
2023-08-11 15:58:06,497 - INFO - data smoothed, shape: (4581, 13)
2023-08-11 15:58:06,970 - INFO - stochastic oscillator collected, shape: (4581, 14)


0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
4576    0.334381
4577    0.195372
4578    0.113966
4579    0.060957
4580    0.000000
Length: 4581, dtype: float64

In [11]:
def get_xy(data):
    x = data.drop(configur.target, axis=1)
    y = data[configur.target]
    # util.logger.info(f"x shape: {x.shape}, y shape: {y.shape}")

    return x,y

def normalize_input(x):
    layer = tf.keras.layers.Normalization(axis=None)
    layer.adapt(x)
    
    return layer(x)

def split_data(x, y):
    n = int(len(x) * configur.train_test_size)
    xtr, xvl = x[:n, :], x[n:, :]
    ytr, yvl = y[:n], y[n:]

    # util.logger.info(f"data splited, train/test shapes: {xtr.shape, ytr.shape}/{xvl.shape, yvl.shape}")
    
    return xtr, xvl, ytr, yvl

x,y = get_xy(repsol.data)
x

,Open,High,Low,Adj Close,Volume,smoothed_series,local_min,local_max,support_line,resistance_line,moving_average_short,moving_average_long,stochastic_oscillator
0,10.660,10.762,10.626,10.722,141705,10.722000,0.000000,0.000000,10.722000,10.722000,0.000000,0.000000,0.000000
1,10.728,10.868,10.706,10.822,16887962,10.780823,0.000000,10.780823,10.735660,10.780823,0.000000,0.000000,0.000000
2,10.838,10.868,10.714,10.746,279069,10.764922,0.000000,0.000000,10.749319,10.784374,0.000000,0.000000,0.000000
3,10.784,10.822,10.714,10.760,345635,10.762979,10.762979,0.000000,10.762979,10.787924,0.000000,0.000000,0.000000
4,10.758,10.850,10.738,10.842,8516401,10.791475,0.000000,10.791475,10.759936,10.791475,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,13.125,13.145,13.110,13.115,12051813,13.144852,0.000000,0.000000,13.104039,13.119300,13.224582,13.302044,0.334381
4577,13.110,13.110,12.965,12.970,239699,13.092397,0.000000,0.000000,13.082698,13.094143,13.189350,13.281395,0.195372
4578,12.965,13.015,12.920,12.990,261616,13.061678,0.000000,0.000000,13.061356,13.068986,13.152259,13.257941,0.113966
4579,12.995,13.020,12.965,12.995,147005,13.041674,0.000000,0.000000,13.040014,13.043829,13.123510,13.232630,0.060957


In [12]:
def normalize_input(x, option="minmax"):
    if option=="minmax":
        scaler = MinMaxScaler()
        x = scaler.fit_transform(x)
    if option=="keras":
        layer = tf.keras.layers.Normalization(axis=None)
        layer.adapt(x)
        x = layer(x)
    
    return x

x = normalize_input(x)
x

array([[0.26021442, 0.26078141, 0.26302191, ..., 0.        , 0.        ,
        0.        ],
       [0.26944475, 0.27511153, 0.27390181, ..., 0.        , 0.        ,
        0.        ],
       [0.28437629, 0.27511153, 0.27498972, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.57309621, 0.56536441, 0.57500338, ..., 0.82141091, 0.82925036,
        0.11396585],
       [0.57716839, 0.56604037, 0.58112334, ..., 0.81961541, 0.82766723,
        0.06095654],
       [0.57716839, 0.56266055, 0.5763634 , ..., 0.81806413, 0.82602961,
        0.        ]])

In [15]:
xtr.reshape(xtr[0], xtr[1], 1)

TypeError: only integer scalar arrays can be converted to a scalar index

In [13]:
xtr, xts, ytr, yts = split_data(x, y)
xtr = xtr.reshape(xtr[0], xtr[1], 1)
xts = xts.reshape(xts[0], xts[1], 1)

TypeError: only integer scalar arrays can be converted to a scalar index

In [41]:






def preprocess(data):
    x,y = get_xy(data)
    x = normalize_input(x)
    xtr, xvl, ytr, yvl = split_data(x, y)

    # reshape (rows, time_step, no_feats)
    xtr = tf.reshape(xtr, (xtr.shape[0], 1, xtr.shape[1]))
    xvl = tf.reshape(xvl, (xvl.shape[0], 1, xvl.shape[1])) 

    # convert to Tensor
    ytr, yvl = tf.constant(ytr.values), tf.constant(yvl.values)

    return xtr, xvl, ytr, yvl

xtr, xvl, ytr, yvl = preprocess(repsol.data)

In [45]:
ytr

<tf.Tensor: shape=(3664,), dtype=float64, numpy=
array([10.72200012, 10.82199955, 10.74600029, ..., 15.25      ,
       15.25500011, 15.14500046])>

In [46]:
tf.reshape(ytr, (ytr.shape[0], 1))

<tf.Tensor: shape=(3664, 1), dtype=float64, numpy=
array([[10.72200012],
       [10.82199955],
       [10.74600029],
       ...,
       [15.25      ],
       [15.25500011],
       [15.14500046]])>

In [47]:
ytr

<tf.Tensor: shape=(3664,), dtype=float64, numpy=
array([10.72200012, 10.82199955, 10.74600029, ..., 15.25      ,
       15.25500011, 15.14500046])>

In [40]:
tf.reshape(xtr, (xtr.shape[0], 1, xtr.shape[1]))

<tf.Tensor: shape=(3664, 1, 13), dtype=float32, numpy=
array([[[-0.09213949, -0.09213927, -0.09213956, ..., -0.09216199,
         -0.09216199, -0.09216199]],

       [[-0.09213935, -0.09213906, -0.09213939, ..., -0.09216199,
         -0.09216199, -0.09216199]],

       [[-0.09213911, -0.09213906, -0.09213938, ..., -0.09216199,
         -0.09216199, -0.09216199]],

       ...,

       [[-0.09212966, -0.09212957, -0.09212984, ..., -0.09212979,
         -0.09212999, -0.09215999]],

       [[-0.09212981, -0.09212977, -0.09212991, ..., -0.09212975,
         -0.09212995, -0.09216005]],

       [[-0.09213012, -0.09213   , -0.09213027, ..., -0.09212974,
         -0.09212992, -0.09216033]]], dtype=float32)>

In [26]:
def normalize_input(x):
    layer = tf.keras.layers.Normalization(axis=None)
    layer.adapt(x)

    return layer(x)

x = normalize_input(x)

In [30]:
x[:1, :]

<tf.Tensor: shape=(1, 13), dtype=float32, numpy=
array([[-0.09213949, -0.09213927, -0.09213956, -0.09213936,  0.20694534,
        -0.09213936, -0.09216199, -0.09216199, -0.09213936, -0.09213936,
        -0.09216199, -0.09216199, -0.09216199]], dtype=float32)>